In [ ]:
import os
import cv2
import numpy as np
from os import path
from tqdm import tqdm
from google.colab import files
from IPython.display import clear_output
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips

In [ ]:
!git clone https://github.com/indianajson/wav2lip-HD.git
basePath = "/content/wav2lip-HD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

!pip install -r requirements.txt
!pip install -U librosa==0.8.1
!pip install basicsr facexlib
!mkdir inputs

!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -P {gfpganFolderName}'/experiments/pretrained_models'

%cd {basePath}

clear_output()

In [ ]:
outputPath = basePath+'/outputs'
inputAudio = 'audio.wav' #@param{type:"string"}
inputAudioPath = basePath + '/inputs/' + inputAudio
inputVideo = 'elon720.mp4' #@param{type:"string"}
inputVideoPath = basePath + '/inputs/'+inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'
model = "wav2lip_gan" #@param ["wav2lip", "wav2lip_gan"] {type:"string"}

if not os.path.exists(outputPath):
  os.makedirs(outputPath)

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath} \
--nosmooth

clear_output()
print("Video is ready in the output folder.")

In [ ]:
inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)

In [ ]:
!cd $gfpganFolderName && python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.4 -s 2 --only_center_face --bg_upsampler None

restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()



#Get FPS of original video for writer
inputVideoPath = outputPath+'/result.mp4'
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)

batch = 0
batchSize = 1300
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end, end="\r")
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
  out = cv2.VideoWriter(processedVideoOutputPath+'/output_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

clear_output()

print("Video upscaling complete.")

In [ ]:
%cd "/content/wav2lip-HD/outputs/"

video_clip = VideoFileClip("/content/wav2lip-HD/outputs/output_0000.mp4")
audio_clip = AudioFileClip(inputAudioPath)

final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile("final_clip" + ".mp4", fps=60, audio_codec="aac")


files.download("/content/final_clip.mp4")

clear_output()
print("Video downloaded!")

In [ ]:
%cd /content/wav2lip-HD/

#@markdown It's recommended to always remove Outputs if you're re-running the code. You may want to preserve the input files if you are only changing one of the two inputs.

removeInputs = True #@param {type:"boolean"}
removeOutputs = True #@param {type:"boolean"}

if removeInputs == True:
  %rm inputs/*
if removeOutputs == True:
  %rm outputs/frames/*
  %rm outputs/restored_imgs/*
  %rm outputs/*

clear_output()

print("Cleared cached files.")